# Blood Pressure Plotting

A short notebook demonstrating how to plot BP readings for a cohort of synthetic patients. Hypertension diagnoses, if present in the patient's history, are also displayed.

To generate a cohort of 100 synthetic patients run the following command inside your local clone of the repo,

`./run_synthea -p 100 --exporter.csv.export true`

In [ ]:
import pandas as pd

# Point this to the csv subfolder of the output directory created by Synthea
output_path = "./uk_synthea/output/csv"

conds = pd.read_csv(f"{output_path}/conditions.csv")
obs = pd.read_csv(f"{output_path}/observations.csv")
meds = pd.read_csv(f"{output_path}/medications.csv")
pats = pd.read_csv(f"{output_path}/patients.csv")
encounters = pd.read_csv(f"{output_path}/encounters.csv")

In [ ]:
import matplotlib.pyplot as plt

def date_format(date_str):
    truncated_str = date_str[:10]
    return pd.to_datetime(truncated_str)


def plot_bp(patient_id, obs_df, conds, encounters):
    ht_conds = conds[conds["DESCRIPTION"] == "Essential hypertension (disorder)"]
    ht_patients = list(ht_conds["PATIENT"])
    if patient_id in ht_patients:
        ht_diagnosis_date = ht_conds[ht_conds["PATIENT"] == patient_id]["START"].iloc[0]
        print(ht_diagnosis_date)
        ht_diagnosis_date = date_format(ht_diagnosis_date)
        print(ht_diagnosis_date)
    else:
        ht_diagnosis_date = None
    sys_values = obs_df[(obs_df["DESCRIPTION"].str.contains("Systolic")) & (obs["PATIENT"] == patient_id)]
    dia_values = obs_df[(obs_df["DESCRIPTION"].str.contains("Diastolic")) & (obs["PATIENT"] == patient_id)]
    fig, ax = plt.subplots(1,1, figsize=(8,4))
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))
    ax.plot(sys_values["DATE"].apply(date_format), sys_values["VALUE"].astype(float), label = "systolic")
    ax.plot(dia_values["DATE"].apply(date_format), dia_values["VALUE"].astype(float), label = "diastolic")

    title = f"BP Measurements for Patient: {patient_id}"

    if ht_diagnosis_date:
        ax.axvline(ht_diagnosis_date, label="HT Diagnosis", c="r", linestyle = "--")
    ax.set_title(title)
    plt.xticks(rotation = 45)
    plt.legend()
    plt.show()

In [ ]:
ht_patients = list(set(conds[conds["DESCRIPTION"] == "Essential hypertension (disorder)"]["PATIENT"]))
patients = list(set(conds["PATIENT"]))

# plot for first 10 patients
for pat in patients[:10]:
    plot_bp(pat, obs, conds, encounters)